<a href="https://colab.research.google.com/github/Jeli04/Sibyl-System/blob/main/SmartChatbot_Sibyl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
import urllib.request 

words = []
classes = []
documents = []
ignore_words = []
'''
Filtered therapy data:
with urllib.request.urlopen("https://raw.githubusercontent.com/Jeli04/ChatBot/main/counsel_no_dupe.json") as url:
    intents = json.loads(url.read().decode())
'''

with urllib.request.urlopen("https://raw.githubusercontent.com/Jeli04/ChatBot/main/sibyl_test_data.json") as url:
    training = json.loads(url.read().decode())

In [3]:
#print(training["data"][0]["questionText"])

for d in training['data']:
    # take each word and tokenize it
    for phrase in d["questionText"]:
      word = nltk.word_tokenize(phrase)
      words.extend(word)
      # adding documents
      documents.append((word, d['intent']))

      # adding classes to our class list
      if d['intent'] not in classes:
          classes.append(d['intent'])

print(classes)
'''
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
'''
print(classes)

['greetings', 'purpose', 'feeling good', 'feeling bad', 'lonely', 'argument', 'depression', 'sad', 'embarrassment', 'nervous', 'talking', 'abuse', 'making friends', 'online friends', 'food', 'TV shows', 'Movies', 'music', 'artist', 'sport', 'creator', 'basketball', 'Soccer', 'Ravi', 'Thanks', 'No', 'goodbyes']
['greetings', 'purpose', 'feeling good', 'feeling bad', 'lonely', 'argument', 'depression', 'sad', 'embarrassment', 'nervous', 'talking', 'abuse', 'making friends', 'online friends', 'food', 'TV shows', 'Movies', 'music', 'artist', 'sport', 'creator', 'basketball', 'Soccer', 'Ravi', 'Thanks', 'No', 'goodbyes']



# Lemmentize
By lemmentizing the words, they are brought to its simpliest forms 

ex: walking, walked, walks all become walk 

In [4]:
print(classes)

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), documents)

print(len(classes), classes)  # Class types of phrases

print(len(words), words) # num of unique words

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

['greetings', 'purpose', 'feeling good', 'feeling bad', 'lonely', 'argument', 'depression', 'sad', 'embarrassment', 'nervous', 'talking', 'abuse', 'making friends', 'online friends', 'food', 'TV shows', 'Movies', 'music', 'artist', 'sport', 'creator', 'basketball', 'Soccer', 'Ravi', 'Thanks', 'No', 'goodbyes']
106 [(['Hello'], 'greetings'), (['Good', 'Morning'], 'greetings'), (['Good', 'Evening'], 'greetings'), (['Good', 'Afternoon'], 'greetings'), (['Hi'], 'greetings'), (['What', "'s", 'up'], 'greetings'), (['Hey'], 'greetings'), (['Yo'], 'greetings'), (['Whats', 'up'], 'greetings'), (['Greetings'], 'greetings'), (['What', 'is', 'your', 'purpose', '?'], 'purpose'), (['What', 'do', 'you', 'do', '?'], 'purpose'), (['Who', 'are', 'you'], 'purpose'), (['What', 'can', 'you', 'do', '?'], 'purpose'), (['What', 'are', 'you', '?'], 'purpose'), (['My', 'day', 'was', 'pretty', 'good', 'today', '!'], 'feeling good'), (['I', 'had', 'a', 'good', 'day', 'today'], 'feeling good'), (['Doing', 'good'],

In [5]:
print(len(words))

156


In [6]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
  # Bag of words 
  bag = []
  # List of tokenized words for the pattern
  pattern_words = doc[0]
  # Lemmetize each word 
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  # Adds words into our bag where 1 means word match is found in the current pattern while 0 is not 
  for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
  # output is a 0 for each tag and 1 for current tag 
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])

print(training)

# Shuffles the training data and turns into np.array
random.shuffle(training)
training = np.array(training)
# Creates train and test lists 
train_x = list(training[:,0])
train_y = list(training[:,1])

print(train_x)
print(train_y)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [7]:
# Creates a model with: First layer 128 neurons, Second layer 64 neurons, and 3rd output layer 
# equal to the number of intents to predict output layer intent with softmax
model = Sequential([Dense(128, input_shape =(len(train_x[0]),), activation = "relu"),
                    Dropout(0.5),
                    Dense(64, activation = "relu"),
                    Dropout(0.5),
                    Dense(len(train_y[0]), activation = "softmax")
])


sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 400, batch_size = 5, verbose = 1)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/400
22/22 [==============================] - 1s 3ms/step - loss: 3.2995 - accuracy: 0.0377
Epoch 2/400
22/22 [==============================] - 0s 3ms/step - loss: 3.2753 - accuracy: 0.0660
Epoch 3/400
22/22 [==============================] - 0s 2ms/step - loss: 3.1608 - accuracy: 0.1226
Epoch 4/400
22/22 [==============================] - 0s 3ms/step - loss: 3.0639 - accuracy: 0.1038
Epoch 5/400
22/22 [==============================] - 0s 3ms/step - loss: 2.9680 - accuracy: 0.2547
Epoch 6/400
22/22 [==============================] - 0s 3ms/step - loss: 2.8455 - accuracy: 0.2358
Epoch 7/400
22/22 [==============================] - 0s 3ms/step - loss: 2.7235 - accuracy: 0.2830
Epoch 8/400
22/22 [==============================] - 0s 2ms/step - loss: 2.6189 - accuracy: 0.2075
Epoch 9/400
22/22 [==============================] - 0s 3ms/step - loss: 2.4670 - accuracy: 0.3585
Epoch 10/400
22/22 [==============================] - 0s 2ms/step - loss: 2.3987 - accuracy: 0.3396
Epoch 11/

In [8]:
print(len(train_x))
test = model.predict(train_x)

106


In [9]:

print(documents[1])
print(classes[1])

(['Good', 'Morning'], 'greetings')
No


In [10]:
model.save("chatbot_counsel_model_test.h5", hist)

# Utlizing the model

In [11]:
from keras.models import load_model
model = load_model('chatbot_counsel_model_test.h5')
import json
import random
with urllib.request.urlopen("https://raw.githubusercontent.com/Jeli04/ChatBot/main/sibyl_test_data.json") as url:
    training = json.loads(url.read().decode())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [12]:
print(training)

{'data': [{'questionText': ['Hello', 'Good Morning', 'Good Evening', 'Good Afternoon', 'Hi', "What's up", 'Hey', 'Yo', 'Whats up', 'Greetings'], 'answerText': ['Hello how are you?', "What's up", "How's your day", 'What can I do for you?', 'Hey'], 'intent': 'greetings'}, {'questionText': ['What is your purpose?', 'What do you do?', 'Who are you', 'What can you do?', 'What are you?'], 'answerText': ["I am Sibyl, a chatbot created with the purpose to answer questions about Social Anxiety Disorder. I can also be something that you can talk too if you are struggling with anything, but I can't gaurantee my advice will help your situation specifcally. Try asking me some fun questions as well! Hopefully I'll be able to understand, I am still learning!"], 'intent': 'purpose'}, {'questionText': ['My day was pretty good today!', 'I had a good day today', 'Doing good', 'Great', 'Good', 'Pretty great'], 'answerText': ["That's good to hear", "That's great!"], 'intent': 'feeling good'}, {'questionTex

In [13]:
# Cleans up the users response 
def clean_up_sentences(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
  return sentence_words

In [14]:
# Bag Of Words (bow)
def bow(sentence, words, show_details = True):
  # Tokenize the patterns 
  sentence_words = clean_up_sentences(sentence)
  # Bag of words
  bag = [0]*len(words)
  for s in sentence_words: 
    for i,w in enumerate(words):
      if w == s:
        # Assigns 1 if the current word is in the vocab position 
        bag[i] = 1
        #if show_details:
          #print("found in bag: %s" % w)
  return np.array(bag)

In [15]:
def predict_class(sentence, model):
  p = bow(sentence, words, show_details=False)
  result = model.predict(np.array([p]))[0]  # Uses the ML model to predict the intent classification results 
  ERROR_THRESHOLD = 0.75  # Threshold to prevent overfitting 
  print("Model Prediction:", result)

  
  results = [[i,r] for i,r in enumerate(result) if r>ERROR_THRESHOLD]
  
  print("test: ", results[0][1])

  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
  '''
  return_list = []
  if result > ERROR_THRESHOLD:
    results = [[i,r] for i,r in enumerate(result)]
    print("test: ", results[0][1])

    results.sort(key=lambda x: x[1], reverse=True)
    for r in results:
      return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
  else:
    return_list.append("unknown", "0%")
  '''
  return return_list

In [16]:
end = True

def getResponse(ints, intents_json):
    intent = ints[0]["intent"]
    list_of_intents = intents_json['data']
    for i in list_of_intents:
        if(i['intent']== intent):
          if(i['intent'] == "goodbyes"):
            print("true")
            end = True

          result = random.choice(i['answerText'])
          break
    return result

In [17]:
def chatbot_response(msg):
  ints = predict_class(msg, model)
  #res = getResponse(msg, ints, training)
  res = getResponse(ints, training)
  return res

In [ ]:
msg = input("This is a test....")
while(msg != "exit" or end == True): # Not detecting the true 
  print(end)
  msg = input("Answer: " + chatbot_response(msg) + "\n")  

# The similarity function is not producing the best results try finidng another method 

This is a test....hello
True
Model Prediction: [7.4171126e-08 3.1831604e-07 1.0166009e-07 7.9323051e-08 2.2960778e-10
 1.1562450e-08 3.7463984e-08 5.9286062e-11 4.6194673e-11 3.6615802e-09
 7.0064601e-08 1.3582945e-09 2.1886033e-09 9.1912642e-07 1.4558394e-06
 2.5287248e-08 3.1653831e-06 9.9999249e-01 4.6355804e-09 5.1049676e-09
 5.2305762e-07 1.0036094e-08 3.5587658e-10 1.2163616e-07 1.0218922e-08
 4.9786945e-07 6.5705419e-09]
test:  0.9999925
Answer: What's up
i am sad
True
Model Prediction: [2.39072895e-13 2.02620681e-11 8.95612057e-11 3.00826258e-14
 1.55949698e-10 1.14921024e-10 3.25077454e-09 5.21284127e-11
 3.47094987e-12 1.42433143e-15 1.47529725e-10 5.79245540e-10
 7.98307340e-12 4.13277497e-15 3.52190607e-12 5.18648346e-10
 1.59588642e-09 1.56549290e-14 9.08171649e-10 6.27945165e-14
 2.92059102e-13 2.97910503e-12 1.61916599e-14 1.41955675e-13
 1.00000000e+00 9.02178434e-15 1.49797794e-14]
test:  1.0
Answer: I can try helping, what's wrong?
i need help making friends
True
Mode

# Plans
- Use monkeylearn to classify intents and tags 
- Use a similarity classifier to determine the similarity betweent the user's input and all the test responses (Change this)


- Find out how to use the dataset to give a appropriate response that matches what the user is asking 
- How to use the answer text 
- How to compare and get the correct response 




# Scope
- Greetings and goodbyes 
- Can pick up a general topics such as depression, alcohol, abuse, or relationship 
- Based on the topic/intent, the bot can give a feedback based on keywords from the response such as cheating, abusive father, self-harm
- Tries to define an intent from the user and searches for a similar response from a category of responses  

# Process
- Gets the intent from the user 
- Searches a set of responses from the intent and finds the most similar one 
- If the user is satisfied or wants to stop, say goodbyes, else continue 